In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
import src.Funciones as fun
from functools import reduce
import operator
from pandas import json_normalize
import numpy as np
from geopy.geocoders import Nominatim
import folium
from folium import Choropleth, Circle, Marker, Icon, Map

In [2]:
geolocator = Nominatim(user_agent="edu")
location = geolocator.geocode("barcelona")
lat = location.latitude
lon = location.longitude

In [3]:
url_query = 'https://api.foursquare.com/v2/venues/explore'

In [4]:
queries_HS = ["Historic Site"]
queries_museum = ["Museum"]
queries_bar = ["Nightclub"]
queries_food = ["Food"]
queries_hotel = ["Hotel"]

In [5]:
tok1= os.getenv("Client_Id")
tok2= os.getenv("Client_Secret")

In [6]:
HS = fun.get_data(lat, lon,url_query,*queries_HS)
museum = fun.get_data(lat, lon,url_query,*queries_museum)
nightclub = fun.get_data(lat, lon,url_query,*queries_bar)
food = fun.get_data(lat, lon,url_query,*queries_food)
hoteles = fun.get_data(lat, lon,url_query,*queries_hotel)

In [7]:
HS = HS.get("Historic Site").get("response").get("groups")[0].get("items")
museum = museum.get("Museum").get("response").get("groups")[0].get("items")
nightclub = nightclub.get("Nightclub").get("response").get("groups")[0].get("items")
food = food.get("Food").get("response").get("groups")[0].get("items")
hoteles = hoteles.get("Hotel").get("response").get("groups")[0].get("items")

In [8]:
df_HS = fun.crear_dataframe(HS)
df_museum = fun.crear_dataframe(museum)
df_nightclub = fun.crear_dataframe(nightclub)
df_food = fun.crear_dataframe(food)
df_hoteles = fun.crear_dataframe(hoteles)

In [9]:
df_HS = df_HS[:10]

In [10]:
df_hoteles = df_hoteles[:10]

In [11]:
df_food = df_food[:10]

In [12]:
df_museum = df_museum[:10]

In [13]:
df_nightclub = df_nightclub[:10]

In [21]:
df_nightclub

,name,latitud,longitud
0,Opium Mar,41.385077,2.174367
1,Macarena Club,41.379434,2.176727
2,Jamboree,41.379947,2.175329
3,The Apartment,41.378998,2.175636
4,Moog,41.378019,2.175063
5,Fantástico Club,41.379029,2.176477
6,Barceloneta Beach Party,41.388409,2.168705
7,BLVD - Boulevard Culture Club,41.378895,2.175253
8,La [2],41.374466,2.169625
9,Nasty Mondays,41.374419,2.169674


## Mapa

In [14]:
map_2 = Map(location=[f'{lat}',f'{lon}'],zoom_start=14.5)

In [15]:
for i, row in df_nightclub.iterrows():
    distrito = {
        "location":[row["latitud"], row["longitud"]],
        "tooltip" : row["name"]}
    icon = Icon(color = "lightred",
                prefix = "fa",
                icon = "glass",
                icon_color = "black")
    Marker(**distrito,icon = icon ).add_to(map_2)

In [16]:
for i, row in df_food.iterrows():
    distrito = {
        "location":[row["latitud"], row["longitud"]],
        "tooltip" : row["name"]}
    icon = Icon(color = "beige",
                prefix = "fa",
                icon = "cutlery",
                icon_color = "black")
    Marker(**distrito,icon = icon ).add_to(map_2)

In [17]:
for i, row in df_hoteles.iterrows():
    distrito = {
        "location":[row["latitud"], row["longitud"]],
        "tooltip" : row["name"]}
    icon = Icon(color = "cadetblue",
                prefix = "fa",
                icon = "h-square",
                icon_color = "white")
    Marker(**distrito,icon = icon ).add_to(map_2)

In [18]:
for i, row in df_HS.iterrows():
    distrito = {
        "location":[row["latitud"], row["longitud"]],
        "tooltip" : row["name"]}
    icon = Icon(color = "lightblue",
                prefix = "fa",
                icon = "eye",
                icon_color = "black")
    Marker(**distrito,icon = icon ).add_to(map_2)

In [19]:
for i, row in df_museum.iterrows():
    distrito = {
        "location":[row["latitud"], row["longitud"]],
        "tooltip" : row["name"]}
    icon = Icon(color = "lightblue",
                prefix = "fa",
                icon = "eye",
                icon_color = "black")
    Marker(**distrito,icon = icon ).add_to(map_2)

In [20]:
map_2